# MK-SQuIT Generation

This notebook details the steps of the generation pipeline in order to produce natural language to SPARQL query pairs.

Some sentence-query examples include:
```
Who is the mother of the director of Pulp Fiction?
SELECT ?end WHERE { [ Pulp Fiction ] wdt:P5 / wdt:P25 ?end . }

Is John Steinbeck the author of Green Eggs and Ham?
ASK { BIND ( [ John Steinbeck ] as ?end ) . [ Green Eggs and Ham ] wdt:P50 ?end . }

How many awards does the producer of Fast and Furious have?
SELECT ( COUNT ( DISTINCT ?end ) as ?endcount ) WHERE { [ Fast and Furious ] wdt:P162 / wdt:P166 ?end . }
```

To provide a brief overview, each query label consists of several components. 
- **SPARQL syntax** typically represented in capital letters (egs. SELECT, ASK, BIND, COUNT, DISTINCT, WHERE).
- **Variables** prefixed by a question mark.
- **Property values** prefixed by P or what we like to call P-value, is an identifier for an entitiy.
- **Item values** prefixed by Q or what we like to call Q-value, which represent the relationship between entities.

The generation pipeline will handle all aspects of the query generation with the exception of item values as building a training set around the sheer number of unique entities would be impractical. Instead, resolving the item value is left as a post-processing step and a simple, but effective method can be found in `txt_2_sparql_gen/utils/entity_resolver.py`.

Do note that manual annotation is required after preprocessing the raw data. Already annotated data is included, and if you'd like to skip that step, feel free to start from the `Generate Type List` section.

In [ ]:
DATA_DIR = "./data"  # Set data path
OUT_DIR = "./out"  # Set output path
ENTITY_ID = "*-5k.json"  # Glob identifier for entity data -> {domain}-5k.json
PROPERTY_ID = "*-props.json"  # Glob identifier for property data -> {domain}-5k.json
PREPROCESSED_ENT_ID = "*-5k-preprocessed.json"  # Glob identifer for preprocesed property data
PREPROCESSED_PROP_ID = "*-props-preprocessed.json"  # Glob identifer for preprocesed property data

# Getting Raw WikiData
To extract raw entity and property data, we can query WikiData and dump them to a folder.

Do note that for the default setings, the naming scheme should follow ENTITY_ID and PROPERTY_ID where the wildcard is replaced by the domain name.

In [ ]:
! python3 -m scripts.gather_wikidata --data-dir $DATA_DIR

# Preprocessing
Raw data must be cleaned and annotated before fed into the pipeline.

Several files are produced:
- `*-5k.json` -> `-5k-preprocessed.json` Entitiy labels are cleaned and aggregated into a single listed field.
- `*-props.json` -> `*-props-preprocessed.jon` Property labels are cleaned and aggregated. Labels are converted into a part-of-speech tag for coherent mapping within a template. For example, "set in location" would be labeled "VERB_NOUN". Lastly, a typing field is added to each property of the format `[domain]->` and must annotated to include `[domain]->[type]`.
- `pos-examples.txt` Samples of part-of-speech tags which are sorted by number of occurances within the data. This is an optional file used to help with template generation.

In [ ]:
! python3 -m scripts.preprocess --data-dir $DATA_DIR --ent-id $ENTITY_ID --prop-id $PROPERTY_ID

# Annotate Data

Each `type` field within `*-props-preprocessed.json` must be annotated with the proper property type before proceeding to the next step.

&emsp;`"type": "{domain}->" should be modified to "type": `"{domain}->{type}"

The type specifies the general category the property falls into. For example, properties "location of" and "location at" could be categorized as "location" whereas "built during" and "created at" could be categorized as "time". To a certain extent, typing is subjective, but allows the pipeline to string together much more coherent statements. For a list of types that we use, refer to the `WH_LABEL_DICT` within `scripts/generate_type_list.py`, which maps a type to a question prefix.

If additional types are required, modifying `WH_LABEL_DICT` may be necessary.

While annotation of generic types and question prefixes require a manual element, they improve the generation of rational queries substantially.

# Generate Type List
Property types, start domains, and type metadata are consolidated into a `type-list-autogenerated.json` file.

In [ ]:
! python3 -m scripts.generate_type_list --data-dir $DATA_DIR --prop-id $PREPROCESSED_PROP_ID

The data is now ready to be fed into the pipeline. The following files should be in the data directory:
- Entity data: `*-5k-preprocessed.json`
- Property data: `*-props-preprocessed.json`
- Part-of-Speech examples (optional): `pos-examples.txt`
- Type metadata list: `type-list-autogenerated.json`

# Generate Dataset

At this point, generation of the pipeline in its current form is simple and the formulation of template variations, insertion of slot values, and generation of query labels is handled by the `FullQueryGenerator`.

Alterations to the template generations and complexity of the queries would be handled here.

In [ ]:
! python3 -m mk_squit.generation.full_query_generator --data-dir $DATA_DIR --prop-id $PREPROCESSED_PROP_ID --ent-id $PREPROCESSED_ENT_ID --out-dir $OUT_DIR

And this will produce data like such:
    
| english                                 | sparql                                                      | unique hash                              |
|-----------------------------------------|-------------------------------------------------------------|------------------------------------------|
| What is the height of Getica's creator? | SELECT ?end WHERE { [ Getica ] wdt:P50 / wdt:P2048 ?end . } | 0ea54cd5187baf7239c8f2023ae33bb3001c5a49 |

# Explore the Dataset

Exploring the dataset is available using Tensorflow Projector at `Explore-Dataset.ipynb`.

# Baseline Model - BART

We also provide a baseline model that is fine tuned from [BART](https://ai.facebook.com/research/publications/bart-denoising-sequence-to-sequence-pre-training-for-natural-language-generation-translation-and-comprehension/). All steps to train and evaluate the model located in the `model` folder.

Alternatively, we have a [tutorial](https://github.com/NVIDIA/NeMo/blob/main/tutorials/nlp/Neural_Machine_Translation-Text2Sparql.ipynb) available through [NVIDIA NeMo](https://github.com/NVIDIA/NeMo). Within our docker container, you will find a localized tutorial of the same notebook in `Neural_Machine_Translation-Text2Sparql.ipynb`.